In [1]:
import os
#Ensure we are at the base level 
os.chdir("../")
%pwd

'C:\\Users\\Markus\\Documents\\Cambridge_Projects\\GroupProject\\conservation_synthesis'

In [12]:
import pathlib
import json
import pandas as pd

In [5]:
crossref_path = pathlib.Path('.') / "data" / f"clean_references_crossref" / "crossref_merged"
openaire_path = pathlib.Path('.') / "data" / f"clean_references_openaire" / "openaire_merged"
CORE_path = pathlib.Path('.') / "data" / f"clean_references_CORE" / "CORE_merged"

goal_path = pathlib.Path('.') / "data" / "merged" 
goal_path.mkdir()

In [31]:
with crossref_path.open("r") as f:
    crossref_data = pd.read_json(f)
with openaire_path.open("r") as f:
    oepnaire_data = pd.read_json(f)
with CORE_path.open("r") as f:
    CORE_data = pd.read_json(f)

In [33]:
crossref_data.head()

,title,authors,doi,publication_date,abstract,repo_identifier,language,publisher,journal_name,journal_volume,journal_issue,issn,url,index
0,The Management of Grass Pastures for Brent Geese,"[J. A.Vickery, W. J.Sutherland, S. J.Lane]",10.2307/2404543,1994-5,None,crossref,EN,JSTOR,NaN,31,2,0021-8901,http://dx.doi.org/10.2307/2404543,0
1,Control of Molinia caerulea on upland moors,"[R. H.Marrs, J. D. P.Phillips, P. A.Todd, J.Gh...",10.1111/j.0021-8901.2004.00901.x,2004-4,None,crossref,EN,Wiley,NaN,41,2,0021-8901,http://dx.doi.org/10.1111/j.0021-8901.2004.009...,2
2,Long-distance relocation of nestboxes reduces ...,"[AlbertoSorace, FabrizioPetrassi, CarloConsiglio]",10.1080/00063650409461343,2004-7,None,crossref,EN,Informa UK Limited,NaN,51,2,0006-3657,http://dx.doi.org/10.1080/00063650409461343,3
3,Reducing the density of breeding gulls influen...,"[S. K.Finney, M. P.Harris, L. F.Keller, D. A.E...",10.1046/j.1365-2664.2003.00810.x,2003-6,None,crossref,EN,Wiley,NaN,40,3,0021-8901,http://dx.doi.org/10.1046/j.1365-2664.2003.008...,4
4,Elements that promote highway crossing structu...,"[WayneMcDonald, Colleen CassadySt Clair]",10.1111/j.1365-2664.2004.00877.x,2004-2-12,None,crossref,EN,Wiley,NaN,41,1,0021-8901,http://dx.doi.org/10.1111/j.1365-2664.2004.008...,5


In [34]:
total_data = pd.concat([crossref_data, oepnaire_data, CORE_data], ignore_index=True, sort=False)

In [37]:
total_data = total_data.sort_values(by="index")

In [48]:
total_data = total_data.reset_index(drop=True)

In [63]:
all_but_authors = list(set(total_data.columns) - set(["authors"]))

In [65]:
total_data = total_data.drop_duplicates(subset = all_but_authors, ignore_index=True)

In [66]:
total_data.head()

,title,authors,doi,publication_date,abstract,repo_identifier,language,publisher,journal_name,journal_volume,journal_issue,issn,url,index
0,The Management of Grass Pastures for Brent Geese,"[J. A.Vickery, W. J.Sutherland, S. J.Lane]",10.2307/2404543,1994-5,None,crossref,EN,JSTOR,NaN,31,2,0021-8901,http://dx.doi.org/10.2307/2404543,0
1,The Management of Grass Pastures for Brent Geese,"[William J. Sutherland, Simon J. Lane, Juliet ...",10.2307/2404543,None,An increasing number of brent geese now feed i...,openaire,EN,JSTOR,The Journal of Applied Ecology,NaN,NaN,NaN,NaN,0
2,Control of Molinia caerulea on upland moors,"[R. H.Marrs, J. D. P.Phillips, P. A.Todd, J.Gh...",10.1111/j.0021-8901.2004.00901.x,2004-4,None,crossref,EN,Wiley,NaN,41,2,0021-8901,http://dx.doi.org/10.1111/j.0021-8901.2004.009...,2
3,Control of Molinia caerulea on upland moors,"[P. A. Todd, M.G. Le Duc, J. Ghorbani, J. D. P...",10.1111/j.0021-8901.2004.00901.x,None,Summary 1 Molinia encroachment has been viewed...,openaire,EN,Wiley,Journal of Applied Ecology,NaN,NaN,NaN,NaN,2
4,Long-distance relocation of nestboxes reduces ...,"[AlbertoSorace, FabrizioPetrassi, CarloConsiglio]",10.1080/00063650409461343,2004-7,None,crossref,EN,Informa UK Limited,NaN,51,2,0006-3657,http://dx.doi.org/10.1080/00063650409461343,3


In [82]:
def merge_rows(rows):
    keys = rows[0].to_dict().keys()
    rows = [r.to_dict() for r in rows]
    merged_row = {}
    for key in keys:
        val = rows[0].get(key)
        for row in rows:
            if key=="authors":
              if val is None or val == []:
                val = row.get(key)
            elif pd.isna(val) or val is None:
                val = row.get(key)
        merged_row[key] = val
    return merged_row

In [83]:
merge_rows([total_data.loc[0], total_data.loc[1]])

{'title': 'The Management of Grass Pastures for Brent Geese',
 'authors': ['J. A.Vickery', 'W. J.Sutherland', 'S. J.Lane'],
 'doi': '10.2307/2404543',
 'publication_date': '1994-5',
 'abstract': 'An increasing number of brent geese now feed inland where they cause economic loss on arable crops. There is, however, little conflict with agriculture when brent geese graze on grass pasture, and encouraging geese onto pasture is a way of reducing conflict between farmers and conservationists. The grazing intensity of wintering brent geese was monitored on areas of grass pasture that had been managed experimentally under different cutting, grazing and fertilizer regimes. There were no significant differences in the grazing intensity of brent geese between treatments in the following experimental comparisons: (a) cutting for silage or sheep grazing or cutting and aftermath grazing with sheep; (b) grazing with cattle or grazing with sheep; or (c) cutting two, three, four or five times',
 'repo_

In [86]:
cleaned_data = []
last_ind = total_data.loc[0]['index']
cur_working_set = []

for ind, row in total_data.iterrows():
    if last_ind == row['index']:
        cur_working_set.append(row)
    else:
        cleaned_data.append(merge_rows(cur_working_set))
        cur_working_set.clear()
        last_ind = row['index']
        cur_working_set.append(row)
        
if cur_working_set != []:
    cleaned_data.append(merge_rows(cur_working_set))

In [87]:
data = pd.DataFrame(cleaned_data)